## [1. 0-1背包问题](https://leetcode.cn/problems/partition-equal-subset-sum/solutions/2785266/0-1-bei-bao-cong-ji-yi-hua-sou-suo-dao-d-ev76/?source=vscode)
#### 状态转移方程：dp[i][j] = dp[i-1][j] or dp[i-1][j-nums[i]]  选或者不选
#### 为了降低空间复杂度，仅用dp[j] = dp[j] or dp[j-nums[i]], 需要j -> v...0 倒叙遍历
#### 为了方便理解可以使用chatgpt举例说明：如果我们先更新 dp[3] = dp[3] or dp[3 - nums[i]]，再更新 dp[4] = dp[4] or dp[4 - nums[i]]，由于 dp[3] 可能已经在本轮更新中通过 nums[i] 变成了 True，那么 dp[4] 也可能会使用这个刚刚更新的 dp[3]，相当于 nums[i] 在同一轮中被用了两次。

# 举例如下：
在0-1背包问题中，如果采用顺序遍历，会导致**同一物品**在**同一轮**被重复使用，进而将问题错误地转化为**完全背包问题**（每个物品可以被无限次使用）。让我们更详细地讨论这种情况。

### 背景：
假设 `nums[i]` 表示第 `i` 个物品的重量，`dp[j]` 表示是否可以通过若干物品组合凑出容量 `j`。

- 在标准的**0-1背包问题**中，每个物品只能被使用一次，因此我们期望在处理物品 `nums[i]` 时，只能通过该物品使用一次的状态转移。
- 如果顺序遍历，会导致状态转移时，某个物品在同一轮中被多次使用，这正是**完全背包问题**的特性。

### 为什么顺序遍历会导致重复使用物品？
为了更详细地理解这个问题，先回顾一下顺序遍历时的流程：

#### 顺序遍历示例：
假设我们有一个物品 `nums[i] = 2`，背包容量为 `5`，初始 `dp` 数组状态如下：
```python
dp = [True, False, False, False, False, False]  # 表示 dp[0] 为 True，其他都为 False
```
现在我们希望使用这个物品 `nums[i]` 更新 `dp` 数组，并且采用顺序遍历的方式来更新：

1. **遍历容量 `j = 2`：**
   - 目标是通过物品 `nums[i]` 来凑出容量 `2`。由于 `dp[2 - nums[i]] = dp[0]` 为 `True`，可以凑出容量 `2`，因此：
   ```python
   dp[2] = dp[2] or dp[0]  # dp[2] = True
   ```
   更新后的 `dp` 数组为：
   ```python
   dp = [True, False, True, False, False, False]
   ```

2. **遍历容量 `j = 3`：**
   - 目标是凑出容量 `3`，我们需要检查 `dp[3 - nums[i]] = dp[1]`。由于 `dp[1] = False`，因此无法凑出容量 `3`：
   ```python
   dp[3] = dp[3] or dp[1]  # dp[3] = False
   ```

3. **遍历容量 `j = 4`：**
   - 目标是凑出容量 `4`，我们需要检查 `dp[4 - nums[i]] = dp[2]`。**此时 `dp[2]` 刚刚在前一步被更新为了 `True`**，表示使用物品 `nums[i]` 可以凑出容量 `2`。由于 `dp[2]` 为 `True`，意味着可以使用这个物品 **再次凑出** 剩余的容量 `2`：
   ```python
   dp[4] = dp[4] or dp[2]  # dp[4] = True
   ```
   更新后的 `dp` 数组为：
   ```python
   dp = [True, False, True, False, True, False]
   ```
   **注意：此时 `dp[4]` 表示通过物品 `nums[i]` **两次**可以凑出容量 `4`，这违反了0-1背包的限制。**

4. **遍历容量 `j = 5`：**
   - 类似地，凑出容量 `5` 需要检查 `dp[5 - nums[i]] = dp[3]`，由于 `dp[3] = False`，因此无法凑出容量 `5`：
   ```python
   dp[5] = dp[5] or dp[3]  # dp[5] = False
   ```

最终，`dp` 数组为：
```python
dp = [True, False, True, False, True, False]
```

### 顺序遍历导致的问题：
通过以上过程可以看到：
- 当我们顺序遍历时，处理容量 `j = 4` 时，检查的 `dp[2]` 已经被当前物品 `nums[i]` 更新为 `True`。因此，容量 `4` 的状态 `dp[4]` 是通过**两次使用物品 `nums[i]`** 产生的。
- **这意味着物品 `nums[i]` 被在同一轮中重复使用了，破坏了0-1背包的要求**，让它变成了完全背包问题。

### 为什么倒序遍历可以避免这个问题？
倒序遍历的本质是在更新容量 `j` 时，确保更新的状态只依赖于上一轮物品的状态，确保物品在每轮只能使用一次。

#### 倒序遍历示例：
我们用倒序遍历重新处理刚才的例子：

1. **遍历容量 `j = 5`：**
   - 目标是凑出容量 `5`，检查 `dp[5 - nums[i]] = dp[3]`，由于 `dp[3] = False`，无法凑出容量 `5`。
   ```python
   dp[5] = dp[5] or dp[3]  # dp[5] = False
   ```

2. **遍历容量 `j = 4`：**
   - 目标是凑出容量 `4`，检查 `dp[4 - nums[i]] = dp[2]`。此时 `dp[2]` 表示通过之前的物品凑出容量 `2`，由于 `dp[2] = True`，因此可以凑出容量 `4`。
   ```python
   dp[4] = dp[4] or dp[2]  # dp[4] = True
   ```

3. **遍历容量 `j = 3`：**
   - 目标是凑出容量 `3`，检查 `dp[3 - nums[i]] = dp[1]`，由于 `dp[1] = False`，无法凑出容量 `3`。
   ```python
   dp[3] = dp[3] or dp[1]  # dp[3] = False
   ```

4. **遍历容量 `j = 2`：**
   - 目标是凑出容量 `2`，检查 `dp[2 - nums[i]] = dp[0]`，由于 `dp[0] = True`，可以凑出容量 `2`。
   ```python
   dp[2] = dp[2] or dp[0]  # dp[2] = True
   ```

最终，`dp` 数组为：
```python
dp = [True, False, True, False, True, False]
```

**倒序遍历时，物品 `nums[i]` 在一轮中只被使用一次，避免了重复使用同一物品的问题，正确地保持了0-1背包问题的性质。**

### 总结：
- **顺序遍历**会导致在更新大容量时使用已经更新过的小容量的状态，从而错误地重复使用了当前物品，使得问题从**0-1背包问题**变成了**完全背包问题**。
- **倒序遍历**确保每次更新当前容量时，依赖的状态是上一轮的物品状态，避免了同一个物品在同一轮中被多次使用，正确地实现了0-1背包的要求。

In [ ]:
### 代码 leetcode416
from typing import List
def canPartition(nums: List[int]) -> bool:
    s = sum(nums)
    if s % 2:
        return False
    s //= 2  # 注意这里把 s 减半了
    n = len(nums)
    f = [[False] * (s + 1) for _ in range(n + 1)]
    f[0][0] = True
    for i, x in enumerate(nums):
        for j in range(s + 1):
            if j >= x:
                f[i + 1][j] = f[i][j - x] or f[i][j]
            else:
                f[i + 1][j] = f[i][j]  
    return f[n][s]

def canPartition(nums: List[int]) -> bool:
    s = sum(nums)
    if s % 2:
        return False
    s //= 2  # 注意这里把 s 减半了
    f = [True] + [False] * s
    s2 = 0
    for i, x in enumerate(nums):
        s2 = min(s2 + x, s)  # 子序列和与s的最小值，前面是因为取不到，后面是没必要，因为超过s就不对了
        for j in range(s2, x - 1, -1):
            f[j] = f[j] or f[j - x]
    return f[s]



In [ ]:
## 322 零钱兑换 完全背包问题，零钱可以被多次使用
from typing import List
def coinChange(coins: List[int], amount: int) -> int:
    n = len(coins)
    dp = [[float('inf')] * (amount + 1) for _ in range(n + 1)]

    dp[0][0] = 0
    for i, x in enumerate(coins):
        for j in range(amount + 1):
            if j >= x:
                dp[i+1][j] = min(dp[i][j], dp[i+1][j - x] + 1) # dp[i+1][j - x] + 1 代表完全背包问题中元素可以被多次选择，与0-1背包问题不同
            else:
                dp[i+1][j] = dp[i][j]
    return dp[n][amount] if dp[n][amount] != float('inf') else -1 

"""
最少硬币个数
dp = [[float('inf')] * (amount + 1) for _ in range(n + 1)]
以1 元和 3 元，目标是凑出总金额为 5 元
dp[1][c]=min(dp[0][c],dp[1][c-1]+1)
for cc in range(c):
    dp[1][1] = 1, dp[1][2] = 2, dp[1][3] = 3, dp[1][4] = 4, dp[1][5] = 5

但如果是 dp[1][c]=min(dp[0][c],dp[0][c-1]+1)
for cc in range(c):
    dp[1][1] = inf, dp[1][2] = min(dp[0][2],dp[0][1]+1) = inf, dp[1][3] = inf, dp[1][4] = inf, dp[1][5] = inf

所以 dp[1][c]=min(dp[0][c],dp[1][c-1]+1), 是代表硬币可以重复使用的完全背包问题
"""

coins = [2]
amount = 3

print(coinChange(coins, amount) )